In [ ]:
from vllm import LLM, SamplingParams
import time
import numpy as np

In [ ]:
with open('long-context.txt', 'r') as f:
    input = f.read()

In [ ]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
llm = LLM(model="THUDM/chatglm3-6b", trust_remote_code=True)
# llm = LLM(model="THUDM/chatglm3-6b", trust_remote_code=True, enable_chunked_prefill=True)

In [ ]:
tokenizer = llm.get_tokenizer()
tokens = tokenizer.encode(input)
tokens += tokens
print(len(tokens))

In [ ]:
import numpy as np
st = time.time()
_, req_token_time, parallel_info = llm.generate(None, sampling_params, prompt_token_ids=[tokens[:1000]] * 4)
for req in req_token_time:
    print((np.array(req_token_time[req]) - st) * 1000000)
print(parallel_info)

In [ ]:
ttft_dict = {req: 0 for req in req_token_time}
for req in req_token_time:
    ttft_dict[req] = (req_token_time[req][0] - st) * 1000000
ttft_dict

In [ ]:
AVG = 5
N_REQ = 2
prefill_list = []
parallel_array = []
for token_len in range(500, 8001, 500):
    avg_prefill = 0
    for i in range(AVG):
        st = time.time()
        _, req_token_time, parallel_info = llm.generate(None, sampling_params, prompt_token_ids=[tokens[:token_len]] * N_REQ)
        for req in req_token_time:
            avg_prefill += (req_token_time[req][0] - st) * 1000000
    prefill_list.append(avg_prefill/AVG/N_REQ)
print(prefill_list)
